As before the code in this notebook should be run in the Algorithm API. 

In the previous tutorials we learned how to download the pipeline data for our backtest. Now our goal is to use the alpha score to construct a target portfolio that maximized the returns based on the alpha scores, while remaining within specific constraints. 

Quantopian has a builtint Optimize API which we can use to order our optimal portfolio. Our list of constraints can be as follows.

- `context.max_leverahe = 1`
- `context.max_pos_size = 0.015`
- `context.max_turnover = 0.95`

In addition to setting the constraints, our portfolio is also exposed to several common risk factors. We will want to limit the impact of these risk factors to our portfolio. We can use Quantopian's builtin risk factor model. <br>

The Risk Model calculates asset exposure to 16 different risk factors: 11 sectors factors and 5 style factors. The Risk Model includes the following factors.

- Sectors
    - Materials
    - Consumer Discretionary
    - Financials
    - Real Estate
    - Consumer Staples
    - Health Care
    - Utilities
    - Telecommunications
    - Energy
    - Industrials
    - Technology
- Style
    - Momentum
    - Size
    - Value
    - Volatility
    - Short term reversal
    

Lets create our Algorithm. We will use both the optimal portfolio as well as the risk model.

In [ ]:
# Portfolio management imports
import quantopian.algorithm as algo
import quantopian.optimize as opt
from quantopian.pipeline.experimental import risk_loading_pipeline

# Pipeline imports

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.filters import QTradableStocksUS


def initialize(context):
    
    # Set the constraints
    context.max_leverage = 1.0
    context.max_pos_size = 0.015
    context.max_turnover = 0.95
    
    # Attach the pipeline
    
    algo.attach_pipeline(
    
        make_pipeline(),
        'data_pipe'
    )
    
    algo.attach_pipeline(
    
        risk_loading_pipeline(),
        'risk_pipe'
    )
    
    # Schedule the rebalance function
    
    algo.schedule_function(
    
        rebalance,
        algo.date_rules.week_start(),
        algo.time_rules.market_open()
    )
    
    
def before_trading_start(context,data):
    
    context.pipeline_data = algo.pipeline_output('data_pipe')
    
    context.risk_factor_betas = algo.pipeline_output('risk_pipe')
    
    
def make_pipeline():
    
    
    sentiment_score = SimpleMovingAverage(
    
        inputs = [stocktwits.bull_minus_bear],
        window_length = 3,
        mask = QTradableStocksUS()
    )
    
    return Pipeline(
    
        columns = {
            
            'sentiment_score':sentiment_score
            
        },screen = sentiment_score.notnull()
    )


def rebalance(context,data):
    
    alpha = context.pipeline_data['sentiment_score']
    
    if not alpha.empty:
        objective = opt.MaximizeAlpha(alpha)
        
        constrain_pos_size = opt.PositionConcentration.with_equal_bounds(
        
            -context.max_pos_size,
            context.max_pos_size
        )
        
        
        max_leverage = opt.MaxGrossExposure(context.max_leverage)
        
        dollar_neutral = opt.DollarNeutral()
        
        max_turnover = opt.MaxTurnover(context.max_turnover)
        
        # Setting the risk factor model
        # Default max 0.2 to sector
        # max 0.4 to Style
        factor_risk_constraints = opt.experimental.RiskModelExposure(
        
            context.risk_factor_betas,
            version = opt.Newest
        
        )
        
        algo.order_optimal_portfolio(
        
            objective = objective,
            constraints = [
                
                constrain_pos_size,
                max_leverage,
                dollar_neutral,
                max_turnover,
                factor_risk_constraints
            ]
        
        
        )
        